In [188]:
import pandas as pd
import numpy as np

In [189]:
#import excel spread sheet of list of violations
violations_list = pd.read_excel("Codes' List of Violations 15 aug 2017.xlsx")

In [190]:
#check out the columns to see how they were importated and how NULL values are treated
violations_list.head(1)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [191]:
#check violation description for duplicates to start cutting down on list
violations_list.VIOLATION_DESC.value_counts(dropna = False)

N                                           137
NaN                                           9
SEWAGE ON GROUND                              2
LANDLORD REGISTRATION                         2
False statement on application                2
CONSIGNMENT SALE                              2
STAGNANT WATER                                2
JUNK TRASH RAT HARBORAGE                      2
Consignment Sale                              2
EXCESS VEGETATION                             2
Distance                                      2
SIGN PERMIT REQUIRED                          2
Hazardous, pathogenic, radioactive waste      2
RAT HARBORAGE GARBAGE                         2
ELECTRICAL SERVICE PANEL                      2
TRASH CANS                                    2
FAULTY PLUMBING                               2
Failure to pay                                2
WATER SYSTEM – CONTAMINATION                  1
Permits to citizen legal resident             1
Occupancy Prohibited                    

In [192]:
#check how complete each column is. VIOLATION_DESC seems to be most complete with only 9 null objects (compared to range index)
violations_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 15 columns):
VIOLATION_ID      928 non-null object
ORG_ID            789 non-null object
VIOLATION_TYPE    788 non-null object
VIOLATION_DESC    925 non-null object
VIOLATION_TEXT    666 non-null object
REMEDIAL_TEXT     345 non-null object
TABLE_NAME        137 non-null datetime64[ns]
EXPIRED_FLAG      651 non-null object
DATE_EXPIRED      2 non-null object
CREATED_BY        650 non-null float64
DATE_CREATED      651 non-null object
MODIFIED_BY       23 non-null object
DATE_MODIFIED     22 non-null datetime64[ns]
FEE_SETUP_ID      0 non-null float64
Unnamed: 14       0 non-null float64
dtypes: datetime64[ns](2), float64(3), object(10)
memory usage: 109.5+ KB


In [193]:
#drop columns: ORG_ID, TABLE_NAME, EXPIRED_FLAG, DATE_EXPIRED, CREATED_BY, DATE_CREATED, MODIFIED_BY, DATE_MODIFIED, FEE_SETUP_ID, Unnamed:14
violations_trimmed = violations_list.drop(["ORG_ID", "TABLE_NAME", "EXPIRED_FLAG", "DATE_EXPIRED", "CREATED_BY", "DATE_CREATED", "MODIFIED_BY", "DATE_MODIFIED", "FEE_SETUP_ID", "Unnamed: 14"], axis=1)

In [194]:
#check to make sure columns were dropped as expected.
violations_trimmed.head(1)

,VIOLATION_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT
0,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS


In [195]:
#VIOLATION_TEXT column contains duplicates, need to clear that up first.
violations_trimmed["VIOLATION_TEXT"].value_counts()

See SW Code                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         4
Section 16.24.340 

In [196]:
violations_trimmed.columns = ["id", "type", "desc", "text", "remedial"]
violations_renamed = violations_trimmed

In [197]:
violations_renamed.head(1)

,id,type,desc,text,remedial
0,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS


In [198]:
violations_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 5 columns):
id          928 non-null object
type        788 non-null object
desc        925 non-null object
text        666 non-null object
remedial    345 non-null object
dtypes: object(5)
memory usage: 36.6+ KB


In [199]:
#used string contains to find all type entries that have "BB", need to use string starts with.
violations_renamed[violations_renamed.type.str.contains("BB", na=False)]

,id,type,desc,text,remedial
406,576,CAAZ_SGN_PROHBBLK,PROHIBITED SIGNS - BLANK,Section 17.32.050 (M) - Prohibited Signs (Blan...,NaN
735,781,BB708020,Sales without a permit,M.C.L 7.08.020: No person shall sell beer with...,NaN
736,782,BB708030B,Off Sale - consume on or near premises,"M.C.L 7.08.030B: A retailer's ""off-sale"" permi...",NaN
737,783,BB708030C,On sale required for on premise consume,"M.C.L 7.08.030C: A retailer's ""on-sale"" permit...",NaN
738,784,BB708030D,Caterer Permit,M.C.L 7.08.030D: A caterer's permit shall be i...,NaN
739,785,BB708030E,Special Event - sales on premises,M.C.L 7.08.030E: A retailer's special events p...,NaN
740,786,BB708040B3,Owners convict beer laws moral turpitude,"M.C.L 7.08.040B3: That no person, firm, corpor...",NaN
741,787,BB708040B5,Owners incarcerated moral turpitude,M.C.L 7.08.040B5: That no person having at lea...,NaN
742,788,BB708040G,Supplemental Application Information,M.C.L 7.08.040G: An applicant or permit holder...,NaN
743,789,BB708040H,False statement on application,M.C.L 7.08.040H: Any applicant making a false ...,NaN


In [200]:
violations_renamed[violations_renamed.type.str.startswith("BB", na=False)]

,id,type,desc,text,remedial
735,781,BB708020,Sales without a permit,M.C.L 7.08.020: No person shall sell beer with...,NaN
736,782,BB708030B,Off Sale - consume on or near premises,"M.C.L 7.08.030B: A retailer's ""off-sale"" permi...",NaN
737,783,BB708030C,On sale required for on premise consume,"M.C.L 7.08.030C: A retailer's ""on-sale"" permit...",NaN
738,784,BB708030D,Caterer Permit,M.C.L 7.08.030D: A caterer's permit shall be i...,NaN
739,785,BB708030E,Special Event - sales on premises,M.C.L 7.08.030E: A retailer's special events p...,NaN
740,786,BB708040B3,Owners convict beer laws moral turpitude,"M.C.L 7.08.040B3: That no person, firm, corpor...",NaN
741,787,BB708040B5,Owners incarcerated moral turpitude,M.C.L 7.08.040B5: That no person having at lea...,NaN
742,788,BB708040G,Supplemental Application Information,M.C.L 7.08.040G: An applicant or permit holder...,NaN
743,789,BB708040H,False statement on application,M.C.L 7.08.040H: Any applicant making a false ...,NaN
744,790,BB708060B,Payment of Privilege Tax,M.C.L 7.08.060B: All permit holders shall pay ...,NaN


In [201]:
#want to get value count in "type" for strings that contain "BB". Possibly use for loop with .apply and .sum functions (https://stackoverflow.com/questions/39403678/finding-the-count-of-a-set-of-substrings-in-pandas-dataframe)

In [202]:
violations_renamed.type.value_counts()

BT575105B3              1
CAP32                   1
WATERWTRHT              1
EXTSWIM                 1
ELECSVCPAN              1
CAAH_LIGHT_HSPC         1
CAP08                   1
SEWRNOFWTR              1
LITHNBILLV              1
CAGK_JTD                1
CAST06                  1
CAAH_EXT_ROD            1
CAAE_ELECSYS_HAZ        1
CAST39                  1
CAHC_ILLEGAL_DUM        1
BT575504D               1
EXTFDWALL               1
CAST18                  1
BT575103A3C             1
LIGHTMAX                1
CAAP_PLUM_RMHSE         1
CAAE_ELECSYS_HAZCERT    1
CAGL_STAG               1
RUBDISGAR               1
BT5755032A              1
HDPM_STAG               1
CAAH_OCCUP_ACCESS       1
CAAH_EXT_DOOR           1
CAAH_EXT_FDWALL         1
CAP33                   1
                       ..
LITVEH                  1
EXTPTSID                1
BBR26                   1
BT575103A4              1
CAST05                  1
BB708140I               1
CAAH_INT_BLDGN          1
CAP16       

In [203]:
violations_renamed[violations_renamed.type.str.startswith("CA", na=False)]

,id,type,desc,text,remedial
7,9,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT
214,381,CAAA_ACCESSORY_STUCT,ACCESSORY STRUCTURES,Section 16.24.330 - Accessory Structures: All ...,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
216,385,CAAA_DUMPSTER,HOURS OF COLLECTION & DUMPSTER REQUIRE,Section 10.20.330 – Dumpsters – Hours of Colle...,NaN
217,386,CAAA_EXHAUST_FAN,EXHAUST FAN DISCHARGE,Section 16.24.330 - Exhaust Fans: Exhaust fan...,NaN
218,387,CAAA_EXTERIOR_REPAIR,EXTERIOR REPAIR,Section 16.24.340 - Exterior Repair: The exter...,NaN
219,388,CAAA_FAILURE_SECURE,FAILURE TO SECURE,Section 16.24.320 - Failure to Secure.,NaN
220,389,CAAA_FENCE_MATERIAL,FENCE MATERIAL,Section 16.04.200 - Fence Materials: Expressl...,NaN
221,390,CAAA_GATE_MAINT,GATE MAINTENANCE,Section 16.24.330 - Gates: Shall be maintaine...,NaN
222,391,CAAA_GRAFFITI,GRAFFITI,Section 16.24.340 - Graffiti: It shall be the ...,NaN


In [205]:
section_pattern = "Section"

In [206]:
violations_section17 = violations_renamed[violations_renamed.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)]

In [207]:
violations_section17

,id,type,desc,text,remedial
0,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,2,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,4,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,5,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,6,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE
5,7,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU
6,8,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG
7,9,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT
8,10,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP
9,11,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC


In [208]:
violations_nosection = violations_renamed.drop(violations_renamed[violations_renamed.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)].index)

In [209]:
violations_nosection

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
106,107,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN


In [210]:
violations_TCA = violations_nosection[violations_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]

In [211]:
violations_TCA

,id,type,desc,text,remedial
106,107,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
376,546,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN
807,853,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN
808,854,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN
809,855,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN
810,856,BT575103A10,Permits to citizen legal resident,"T.C.A 57-5-103(a)(10): After July 1, 2015, a c...",NaN
811,857,BT575103A2,Permit to business owner,T.C.A 57-5-103(a)(2): Permits shall be issued ...,NaN
812,858,BT575103A3A,Cannot transfer ownership,T.C.A 57-5-103(a)(3)(A): A permit shall be val...,NaN
813,859,BT575103A3B,Permit for single location only,T.C.A 57-5-103(a)(3)(B): A permit shall be val...,NaN
814,860,BT575103A3C,Business Name same as Permit Name,T.C.A 57-5-103(a)(3)(C): A permit shall be val...,NaN


In [212]:
violations_noTCA = violations_nosection.drop(violations_nosection[violations_nosection.text.str.match(r"(T.C.A|Tennessee Code Annotated)", na=False)].index)

In [213]:
violations_noTCA

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN


In [214]:
violations_noTCA[violations_noTCA.text.str.contains("M.C.L", na=False)]

,id,type,desc,text,remedial
735,781,BB708020,Sales without a permit,M.C.L 7.08.020: No person shall sell beer with...,NaN
736,782,BB708030B,Off Sale - consume on or near premises,"M.C.L 7.08.030B: A retailer's ""off-sale"" permi...",NaN
737,783,BB708030C,On sale required for on premise consume,"M.C.L 7.08.030C: A retailer's ""on-sale"" permit...",NaN
738,784,BB708030D,Caterer Permit,M.C.L 7.08.030D: A caterer's permit shall be i...,NaN
739,785,BB708030E,Special Event - sales on premises,M.C.L 7.08.030E: A retailer's special events p...,NaN
740,786,BB708040B3,Owners convict beer laws moral turpitude,"M.C.L 7.08.040B3: That no person, firm, corpor...",NaN
741,787,BB708040B5,Owners incarcerated moral turpitude,M.C.L 7.08.040B5: That no person having at lea...,NaN
742,788,BB708040G,Supplemental Application Information,M.C.L 7.08.040G: An applicant or permit holder...,NaN
743,789,BB708040H,False statement on application,M.C.L 7.08.040H: Any applicant making a false ...,NaN
744,790,BB708060B,Payment of Privilege Tax,M.C.L 7.08.060B: All permit holders shall pay ...,NaN


In [215]:
violations_MCL = violations_noTCA[violations_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]

In [216]:
violations_MCL

,id,type,desc,text,remedial
481,627,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN
482,628,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN
483,629,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN
484,630,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN
485,631,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN
486,632,CAGF_LPU,LITTER ON PUBLIC PROPERTY,MCL Chapter 10.24.140 - Litter on vacant lots....,NaN
487,633,CAGG_LVH,LITTER FROM VEHICLES,MCL Chapter 10.24.150 - Litter from vehicles--...,NaN
488,634,CAGH_HWG,EXCESS VEGETATION,MCL Chapter 10.26.010 – Every premise includin...,NaN
489,635,CAGI_GCAN,GARBAGE APPROVED CONTAINERS,MCL Chapter 10.32.120 - All garbage and trash ...,NaN
490,636,CAGJ_GT_RATS,RAT HARBORAGE GARBAGE,MCL Chapter 10.32.130 – Permitting garbage and...,NaN


In [217]:
violations_noMCL = violations_noTCA.drop(violations_noTCA[violations_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)].index)

In [218]:
violations_noMCL

,id,type,desc,text,remedial
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN
